In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


In [2]:
# A cell class for constraint propagation
class Cell:
    def __init__(self, current, r, c, found, value, possible_numbers):
        self.current = current
        self.r = r
        self.c = c
        self.found = found
        self.value = value
        self.possible_numbers = possible_numbers


# Checks the initial state of the sudoku
def check_initial_state(sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            number = sudoku[r][c]
            if number != 0:
                sudoku[r][c] = -1
                if not is_ok(r, c, number, sudoku):
                    return False
                sudoku[r][c] = number
    return True


# Checks if the number is in that row of the sudoku
def is_in_row(row, number, sudoku):
    for i in range(0, 9):
        if sudoku[row][i] == number:
            return True
    return False


# Checks if the number is in that col of the sudoku
def is_in_col(col, number, sudoku):
    for i in range(0, 9):
        if sudoku[i][col] == number:
            return True
    return False


# Checks if the number is in the smaller 3x3 grid of that row and col of the sudoku
def is_in_box(row, col, number, sudoku):
    r = row - row % 3
    c = col - col % 3

    for i in range(r, r + 3):
        for j in range(c, c + 3):
            if sudoku[i][j] == number:
                return True

    return False


# Returns true if the number can be placed at that row and col of the sudoku
def is_ok(row, col, number, sudoku):
    return (
        not (is_in_row(row, number, sudoku)) and not (is_in_col(col, number, sudoku)) and 
        not (is_in_box(row, col, number, sudoku))
    )

def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """

    ### YOUR CODE HERE

    if not check_initial_state(sudoku):
        return np.full((9, 9), -1)

    if solve(sudoku):
        return sudoku
    else:
        return np.full((9, 9), -1)


# SEARCH ALGORITHM

def solve(sudoku):
    # Constrain the search space
    reduce_sudoku(sudoku)

    # In case during constraint propagation an invalid value has been put in
    if not check_initial_state(sudoku):
        return False

    # Loop through every cell in the sudoku
    for r in range(0, 9):
        for c in range(0, 9):
            # If it's 0
            if sudoku[r][c] == 0:
                for number in range(1, 10):
                    # Check if any numbers can be put in the sudoku
                    if is_ok(r, c, number, sudoku):
                        # Create copy of current state of sudoku for backtracking
                        old_sudoku = copy_sudoku(sudoku)

                        # If the number works put it in the soduku as a guess
                        sudoku[r][c] = number

                        # Solve the current state of the sudoku
                        if solve(sudoku):
                            return True
                        # Backtrack if it doesn't work and reset values
                        else:
                            backtrack(sudoku, old_sudoku)
                            # Free up memory space
                            del old_sudoku

                # If no possible numbers return false
                return False
    # If reaches the end sudoku is solved
    return True

# MAKE COPY OF SUDOKU

# This is slightly faster than copy.deepcopy
def copy_sudoku(sudoku):
    copied_sudoku = np.empty([9, 9], dtype=int)
    for r in range(0, 9):
        for c in range(0, 9):
            copied_sudoku[r][c] = sudoku[r][c]
    return copied_sudoku

# BACKTRACK

# So that the search algorithm doesn't return the wrong object
def backtrack(sudoku, old_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            sudoku[r][c] = old_sudoku[r][c]

# TURN NUMBER BASED SUDOKU TO CELL BASED SUDOKU

def make_cell_sudoku(sudoku):
    cell_sudoku = np.empty([9, 9], dtype=Cell)
    solved_values = 0

    for r in range(0, 9):
        for c in range(0, 9):
            # Value needs to be found
            if sudoku[r][c] == 0:
                numbers = []
                for i in range(1, 10):
                    numbers.append(i)
                temp = Cell(False, r, c, False, 0, numbers)
                cell_sudoku[r][c] = temp
            # Value is already found
            else:
                numbers = []
                temp = Cell(False, r, c, True, sudoku[r][c], numbers)
                cell_sudoku[r][c] = temp
                solved_values += 1

    return cell_sudoku, solved_values


# CONSTRAINT PROPAGATION

def reduce_sudoku(sudoku):
    cell_sudoku, solved_values = make_cell_sudoku(sudoku)

    stalled = False
    while not stalled:
        solved_values_before = solved_values

        # Row
        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, False)
        eliminate(cell_sudoku)
        only_choice_row(cell_sudoku)

        # Col
        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, False)
        eliminate(cell_sudoku)
        only_choice_col(cell_sudoku)

        # Box
        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, False)
        eliminate(cell_sudoku)
        only_choice_box(cell_sudoku)

        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, True)
        eliminate(cell_sudoku)

        # If we can't do anything then we are stalled
        if not stalled:
            stalled = solved_values == solved_values_before


# FIND CELLS WITH ONLY ONE POSSIBLE VALUE

def single_possible_value_finder(cell_sudoku, sudoku, solved_values, check_state):
    for r in range(0, 9):
        for c in range(0, 9):
            if len(cell_sudoku[r][c].possible_numbers) == 1:
                cell_sudoku[r][c].found = True
                cell_sudoku[r][c].value = cell_sudoku[r][c].possible_numbers[0]
                cell_sudoku[r][c].possible_numbers.clear()

                sudoku[r][c] = cell_sudoku[r][c].value

                solved_values += 1

                # Will be true for last call of this method
                if check_state:
                    # If our current state is not okay
                    if not check_initial_state(sudoku):
                        # Set stalled to true
                        return True, solved_values

                return False, solved_values

    # Means sudoku is solved
    return True, solved_values


# ELIMINATE NOT POSSIBLE NUMBERS FROM CELLS

def eliminate(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            if cell_sudoku[r][c].found == False:
                already_have_numbers = []

                # Remove non possible numbers from the row and column
                for i in range(0, 9):
                    if cell_sudoku[r][i].found == True:
                        already_have_numbers.append(cell_sudoku[r][i].value)
                    if cell_sudoku[i][c].found == True:
                        already_have_numbers.append(cell_sudoku[i][c].value)

                # Remove non possible numbers from the 3x3 grid
                r0 = r - r % 3
                c0 = c - c % 3
                for i in range(r0, r0 + 3):
                    for j in range(c0, c0 + 3):
                        if cell_sudoku[i][j].found == True:
                            already_have_numbers.append(cell_sudoku[i][j].value)

                for i in range(0, len(already_have_numbers)):
                    if already_have_numbers[i] in cell_sudoku[r][c].possible_numbers:
                        cell_sudoku[r][c].possible_numbers.remove(already_have_numbers[i])

                already_have_numbers.clear()


# CHECK THE ONLY SUITABLE NUMBERS FOR A CELL

# The only choice for the row
def only_choice_row(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            should_continue = True
            # If current cell not solved
            if cell_sudoku[r][c].found == False:
                cell_sudoku[r][c].current = True
                # Check it's possible numbers
                for i in range(0, len(cell_sudoku[r][c].possible_numbers)):
                    if should_continue:
                        # Will look at every non-found cell's possible numbers in this row
                        for row in range(0, 9):
                            if (cell_sudoku[r][row].found == False) and (cell_sudoku[r][row].current == False):
                                # If they could both be i break 
                                if cell_sudoku[r][c].possible_numbers[i] in cell_sudoku[r][row].possible_numbers:
                                    break

                            # If no other cells in the row can be i, then this cell has to be i
                            if row == 8:
                                cell_sudoku[r][c].current = False
                                cell_sudoku[r][c].possible_numbers = [cell_sudoku[r][c].possible_numbers[i]]
                                should_continue = False
                    else:
                        break
                cell_sudoku[r][c].current = False

# The only choice for the column
def only_choice_col(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            should_continue = True
            # If current cell not solved
            if cell_sudoku[r][c].found == False:
                cell_sudoku[r][c].current = True
                # Check it's possible numbers
                for i in range(0, len(cell_sudoku[r][c].possible_numbers)):
                    if should_continue:
                        # Will look at every non-found cell's possible numbers in this column
                        for col in range(0, 9):
                            if (cell_sudoku[col][c].found == False) and (cell_sudoku[col][c].current == False):
                                # If they could both be i break 
                                if cell_sudoku[r][c].possible_numbers[i] in cell_sudoku[col][c].possible_numbers:
                                    break

                            # If no other cells in the column can be i, then this cell has to be i
                            if col == 8:
                                cell_sudoku[r][c].current = False
                                cell_sudoku[r][c].possible_numbers = [cell_sudoku[r][c].possible_numbers[i]]
                                should_continue = False
                    else:
                        break
                cell_sudoku[r][c].current = False

# The only choice for the box
def only_choice_box(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            should_continue2 = True
            # If current cell not solved
            if cell_sudoku[r][c].found == False:
                cell_sudoku[r][c].current = True
                # Check it's possible numbers
                for i in range(0, len(cell_sudoku[r][c].possible_numbers)):
                    if should_continue2:
                        r0 = r - r % 3
                        c0 = c - c % 3
                        should_continue = True
                        # Will look at every non-found cell's possible numbers in this box
                        for row in range(r0, r0 + 3):
                            for col in range(c0, c0 + 3):
                                if should_continue:
                                    # If they could both be i break 
                                    if (
                                        (cell_sudoku[row][col].found == False) 
                                        and (cell_sudoku[row][col].current == False)
                                    ):
                                        if (
                                            cell_sudoku[r][c].possible_numbers[i] in 
                                            cell_sudoku[row][col].possible_numbers
                                        ):
                                            should_continue = False
                                            break
                                    # If no other cells in the box can be i, then this cell has to be i
                                    if (row == r + 2) and (col == c + 2):
                                        cell_sudoku[r][c].current = False
                                        cell_sudoku[r][c].possible_numbers = [cell_sudoku[r][c].possible_numbers[i]]
                                        should_continue2 = False

                    else:
                        break
                cell_sudoku[r][c].current = False

In [3]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.016492624000000067 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]


This is your solution for easy sudoku number 5
[[6 4 5 3 2 8 7 1 9]
 [3 8 7 9 1 6 4 5 2]
 [2 9 1 4 5 7 6 3 8]
 [5 6 3 2 9 1 8 7 4]
 [9 7 8 6 4 5 1 2 3]
 [1 2 4 8 7 3 5 9 6]
 [7 3 9 5 6 4 2 8 1]
 [8 5 6 1 3 2 9 4 7]
 [4 1 2 7 8 9 3 6 5]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.016814201999999945 seconds to solve.

This is easy sudoku number 6
[[6 4 1 3 8 2 0 7 9]
 [5 2 9 4 1 7 8 6 3]
 [3 8 7 0 6 0 2 4 1]
 [8 1 2 6 7 3 4 9 5]
 [4 9 0 8 2 1 0 3 6]
 [7 3 6 5 9 4 1 8 2]
 [9 6 4 1 5 8 3 2 7]
 [1 7 8 2 3 6 9 5 4]
 [2 5 3 7 4 9 6 1 8]]
This is your solution for easy sudoku number 6
[[6 4 1 3 8 2 5 7 9]
 [5 2 9 4 1 7 8 6 3]
 [3 8 7 9 6 5 2 4 1]
 [8 1 2 6 7 3 4 9 5]
 [4 9 5 8 2 1 7 3 6]
 [7 3 6 5 9 4 1 8 2]
 [9 6 4 1 5 8 3 2 7]
 [1 7 8 2 3 6 9 5 4]
 [2 5 3 7 4 9 6 1 8]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.014932156999999835 seconds to solve.

This is easy sudoku number 7
[[1 4 8 2 6 0 3 7 5]
 [6 0 0 7 3 5 1 4 8]
 [3 5 7 4 8 1 0 9 2]
 [5

This is your solution for medium sudoku number 4
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.05775186100000007 seconds to solve.

This is medium sudoku number 5
[[0 8 5 0 1 3 0 0 9]
 [6 3 4 0 0 2 1 7 5]
 [0 2 0 5 7 4 0 3 0]
 [2 4 8 3 6 7 9 5 1]
 [9 6 0 4 5 8 0 2 3]
 [3 5 7 2 0 0 4 8 0]
 [5 7 3 1 0 0 8 9 2]
 [4 9 6 0 2 5 3 1 0]
 [8 1 2 0 3 9 5 6 4]]
This is your solution for medium sudoku number 5
[[7 8 5 6 1 3 2 4 9]
 [6 3 4 9 8 2 1 7 5]
 [1 2 9 5 7 4 6 3 8]
 [2 4 8 3 6 7 9 5 1]
 [9 6 1 4 5 8 7 2 3]
 [3 5 7 2 9 1 4 8 6]
 [5 7 3 1 4 6 8 9 2]
 [4 9 6 8 2 5 3 1 7]
 [8 1 2 7 3 9 5 6 4]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.06664894500000007 seconds to solve.

This is me

This is your solution for hard sudoku number 4
[[9 4 7 2 8 3 6 5 1]
 [1 6 5 7 9 4 2 8 3]
 [3 8 2 5 6 1 7 9 4]
 [8 5 6 4 7 2 3 1 9]
 [7 2 3 9 1 5 8 4 6]
 [4 9 1 8 3 6 5 2 7]
 [5 7 8 6 4 9 1 3 2]
 [6 3 4 1 2 8 9 7 5]
 [2 1 9 3 5 7 4 6 8]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.6551063150000012 seconds to solve.

This is hard sudoku number 5
[[0 0 2 0 6 0 0 3 0]
 [0 5 0 0 1 0 0 0 7]
 [0 0 0 4 0 0 0 0 0]
 [1 0 0 0 0 8 0 0 0]
 [5 0 4 1 2 0 0 0 6]
 [0 6 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 7 0 0]
 [0 9 0 0 0 2 5 0 8]
 [0 0 0 0 5 0 0 6 0]]
This is your solution for hard sudoku number 5
[[8 1 2 7 6 5 9 3 4]
 [4 5 3 2 1 9 6 8 7]
 [9 7 6 4 8 3 1 5 2]
 [1 2 9 6 3 8 4 7 5]
 [5 3 4 1 2 7 8 9 6]
 [7 6 8 5 9 4 2 1 3]
 [3 8 5 9 4 6 7 2 1]
 [6 9 1 3 7 2 5 4 8]
 [2 4 7 8 5 1 3 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 3.015347193 seconds to solve.

This is hard sudoku number 6
[[0 8 0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2]
 [0 5 0 7 0 8 0 9 0]
 [0 4 0 0 2 0 